In [1]:
from scripts.segment import *
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from functools import partial
from multiprocessing import Pool

In [2]:
# grid_size = 10
# num_simulation = 10
# num_test = 20
# fail_rates = np.linspace(0, 1, num=num_test)



# pool = Pool(processes=num_test)


In [3]:
def generate_sim_report_topo(grid_size,num_simulation,fail_rate):
    report = SegmentReport(grid_size)
    valves = generate_valves_grid(grid_size)
    valve_register = ValveRegister()
    register_valves(valves,valve_register)
    for i in range(num_simulation):
        valves_dict = generate_valves_dict(valve_register,x,fail_rate)
        segments = simulate_segments(grid_size,valves_dict)
        report = update_segment_report(segments,report)
    return report

def generate_nx_config(valve_register,x):
    nid2valve = valve_register.nid2v
    vids2fail = []
    for _,valves in nid2valve.items():
        removed_valves = np.random.choice(valves, x, replace=False)
        for removed_valve in removed_valves:
            vids2fail.append(removed_valve.vid)
    
    vids = valve_register.vid2v.keys()
    remained_vids = list(set(vids)-set(vids2fail))

    return vids2fail,remained_vids

def generate_vids2fail(vids,fail_rate):
    num_failed_valves = int(fail_rate*len(vids))
    rand_fvids = list(np.random.choice(vids, num_failed_valves, replace=False))
    return rand_fvids

In [35]:
from scripts.node import *

In [36]:

        
grid_size = 3
valves = generate_valves_grid(grid_size)
valve_register = ValveRegister()
register_valves(valves,valve_register)

nids2node = get_nid2nodes(grid_size,valve_register.nid2v)
node = nids2node[0]

print (node)

nid: 0, ntype: NodeType.UPPER_LEFT,        nneighbors: left: None, right: 1, up None, down 3,        pneighbors: [None, 0, None, 6],        vneighbors: left: None, right: valve with vid 0, links to nid 0 and pid 0, fail status 0 , up: None, down: valve with vid 12, links to nid 0 and pid 6, fail status 0 ,        degree: 2


In [37]:
from scripts.topology_operations import *

In [30]:
set(range(10))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [34]:
mesh.degree_distribution

{0: [],
 1: [],
 2: [0, 4, 20, 24],
 3: [1, 2, 3, 5, 9, 15, 21, 22, 23],
 4: [12, 16, 17, 18, 14],
 5: [10],
 6: [13, 6],
 7: [],
 8: [],
 9: []}

In [43]:
from scripts.graph import * 

In [44]:
def check_left_right(left_node,right_node):
    left_node_valves = left_node.valve_neighbors
    right_node_valves = right_node.valve_neighbors
    if (not left_node_valves.right.fail) and (not right_node_valves.left.fail):
        return True
    return False

def check_up_down(up_node,down_node):
    up_node_valves = up_node.valve_neighbors
    down_node_valves = down_node.valve_neighbors
    if (not up_node_valves.down.fail) and (not down_node_valves.up.fail):
        return True
    return False
        
def check_direct_linkage(node1,node2):
    n1_neighbors = node1.get_node_neighbor_ids()
    nid2 = node2.nid
    if nid2 not in n1_neighbors:
        return False
    
    if n1_neighbors[0] == nid2:
        return check_left_right(node2,node1)
    if n1_neighbors[1] == nid2:
        return check_left_right(node1,node2)
    if n1_neighbors[2] == nid2:
        return check_up_down(node2,node1)
    if n1_neighbors[3] == nid2:
        return check_up_down(node1,node2)
    
        
grid_size = 3
valves = generate_valves_grid(grid_size)
valve_register = ValveRegister()
register_valves(valves,valve_register)
nids2node = get_nid2nodes(grid_size,valve_register.nid2v)

check_direct_linkage(nids2node[1],nids2node[0])

True

In [45]:
def get_direct_link_nodes(node,nids2node):
    neighbors = node.get_node_neighbor_ids()
    linked_nodes = []
    for nid in neighbors:
        if (nid!=None):
            neighbor_node = nids2node[nid]
            if check_direct_linkage(node,neighbor_node):
                linked_nodes.append(neighbor_node)
    return linked_nodes
        
nodes = get_direct_link_nodes(nids2node[3],nids2node)
for node in nodes:
    print (node)

# nodes = get_direct_link_nodes(nids2node[4],nids2node)
# for node in nodes:
#     print (node)

nid: 4, ntype: NodeType.NORMAL,        nneighbors: left: 3, right: 5, up 1, down 7,        pneighbors: [2, 3, 8, 9],        vneighbors: left: valve with vid 5, links to nid 4 and pid 2, fail status 0 , right: valve with vid 6, links to nid 4 and pid 3, fail status 0 , up: valve with vid 17, links to nid 4 and pid 8, fail status 0 , down: valve with vid 18, links to nid 4 and pid 9, fail status 0 ,        degree: 4
nid: 0, ntype: NodeType.UPPER_LEFT,        nneighbors: left: None, right: 1, up None, down 3,        pneighbors: [None, 0, None, 6],        vneighbors: left: None, right: valve with vid 0, links to nid 0 and pid 0, fail status 0 , up: None, down: valve with vid 12, links to nid 0 and pid 6, fail status 0 ,        degree: 2
nid: 6, ntype: NodeType.LOWER_LEFT,        nneighbors: left: None, right: 7, up 3, down None,        pneighbors: [None, 4, 7, None],        vneighbors: left: None, right: valve with vid 8, links to nid 6 and pid 4, fail status 0 , up: valve with vid 15, lin

In [46]:
def assemble_adjacency_mtx_nodes(nids2node):
    tot_nodes = len(nids2node)
    rows,cols = [],[]
    for nid,node in nids2node.items():
        connected_nodes = get_direct_link_nodes(node,nids2node)
        for connected_node in connected_nodes:
            rows.append(nid)
            cols.append(connected_node.nid)

    vals = 1+np.zeros(len(rows))
    A = csc_matrix((vals, (rows, cols)), shape=(tot_nodes,tot_nodes))
    return A

A = assemble_adjacency_mtx_nodes(nids2node)
print (A.todense())

[[0. 1. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 1. 0. 1. 0. 0.]
 [0. 1. 0. 1. 0. 1. 0. 1. 0.]
 [0. 0. 1. 0. 1. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 1. 0.]]


In [47]:
def create_reversed_valves(valves):
    reversed_valves = []
    for v in valves:
        valve = v.copy()
        valve.fail = ~valve.fail
        reversed_valves.append(valve)
    return reversed_valves

In [48]:
grid_size = 3
valves = generate_valves_grid(grid_size)
valve_register = ValveRegister()
register_valves(valves,valve_register)
nids2node = get_nid2nodes(grid_size,valve_register.nid2v)
len(nids2node)
# reversed_valve_dict = reverse_valve_fail(valve_register.vid2v)
# nids2node = get_nid2nodes(grid_size,valve_register.vid2v)
# A = assemble_adjacency_mtx(grid_size,reversed_valve_dict)

9

In [69]:
def node_pair2pid(node1,node2):
    pids1 = set(node1.get_pipe_neighbor_ids())
    pids2 = set(node2.get_pipe_neighbor_ids())
    pids = pids1.intersection(pids2)
    
    pid = None 
    for p in pids:
        if p!= None:
            pid = p
    return pid
 
node1 = nids2node[1]
node2 = nids2node[4]

node_pair2pid(node1,node2)

8

In [70]:
def convert_npairs2pids(node_pairs,nids2node):
    pids = []
    for nid1,nid2  in node_pairs:
        n1,n2 = nids2node[nid1],nids2node[nid2]
        pids.append(node_pair2pid(n1,n2))
    return pids
        
        

    
    

In [71]:
def get_bfs_edges(A,nids2node):
    component,edges = bsf_tree(0,A)
    edges = convert_npairs2pids(edges,nids2node) 
    return edges

get_bfs_edges(A,nids2node)

[0, 6, 2, 7, 4, 9, 5, 11, 10, 3, 8, 1]

In [87]:
class Mesh(object):
    def __init__(self,grid_size):
        self.grid_size = grid_size
        self.nids = set(range(grid_size*grid_size))
        
        self.valve_register = self._create_valveNregister()
        self.nid2nodes = self._create_nid2nodes()
        self.contraction_reg = ContractionRegister()
        self.degree_distribution = self._init_degree_distribution()
        
        self.adj_mtx_nodes = assemble_adjacency_mtx_nodes(self.nid2nodes)
        self.bfs_edges = get_bfs_edges(self.adj_mtx_nodes,self.nid2nodes)
#         self.deletion_reg = DeletionRegister()
        
    def _create_valveNregister(self):
        valves = generate_valves_grid(self.grid_size)
        valve_register = ValveRegister()
        register_valves(valves,valve_register)
        return valve_register
    
    def _create_nid2nodes(self):
        return get_nid2nodes(self.grid_size,self.valve_register.nid2v)
    
    def _init_degree_distribution(self):
        degrees = {}
        for i in range(10):
            degrees[i] = []
        return degrees
    
    def perform_contractions(self,identify_pairs_op):
        self.contraction_reg = register_contractions(identify_pairs_op,self.contraction_reg)
    
    def deletions(self):
        return None
    
    def update_degree_distribution(self):    
        self._init_degree_distribution()
        normal_nids = self.nids-self.contraction_reg.contracted_nids
        for normal_nid in normal_nids:
            node = self.nid2nodes[normal_nid]
            degree = node.degree()
            self.degree_distribution[degree].append(normal_nid)
        
        for cn1,cn2 in self.contraction_reg.npairs:
            degree = cn1.degree()+cn2.degree()-2
            self.degree_distribution[degree].append(cn1.nid)
        
        
        

In [88]:
grid_size = 5
contraction_ratio = 0.2
mesh = Mesh(grid_size)

nids2identify = get_nids2identify(grid_size,contraction_ratio)
identify_pairs_op =  get_identify_pairs_op(nids2identify,mesh.nid2nodes)
mesh.perform_contractions(identify_pairs_op)
mesh.update_degree_distribution()
print(mesh.contraction_reg)

npairs: [(<__main__.Node object at 0x7fc125cd7f60>, <__main__.Node object at 0x7fc125d526a0>), (<__main__.Node object at 0x7fc125cd78d0>, <__main__.Node object at 0x7fc125cd75c0>), (<__main__.Node object at 0x7fc125d52320>, <__main__.Node object at 0x7fc125cd77f0>), (<__main__.Node object at 0x7fc125d52550>, <__main__.Node object at 0x7fc125d526a0>)], vpairs: [(<scripts.valves.Valve object at 0x7fc125d78d30>, <scripts.valves.Valve object at 0x7fc125d78a90>), (<scripts.valves.Valve object at 0x7fc125d78240>, <scripts.valves.Valve object at 0x7fc125d784e0>), (<scripts.valves.Valve object at 0x7fc125d78940>, <scripts.valves.Valve object at 0x7fc125d78780>), (<scripts.valves.Valve object at 0x7fc125d7e2b0>, <scripts.valves.Valve object at 0x7fc125d7e780>)],        trivial_pids: [39, 30, 27, 19],contracted_nids: {12, 16, 17, 19, 21, 23, 24}


In [77]:
mesh.degree_distribution

{0: [],
 1: [],
 2: [4, 20, 24],
 3: [2, 3, 14, 15, 19, 21, 22, 23, 0],
 4: [6, 7, 11, 12, 13, 18, 5],
 5: [9],
 6: [17],
 7: [],
 8: [],
 9: []}

In [85]:
class Node(object):
    def __init__(self,nid,node_type_checker,grid_size):
        self.nid = nid
        self.node_type = node_type_checker.get_node_type(nid)
        self.node_neighbors = NodeNeighbors(nid,grid_size)
        self.valve_neighbors = ValveNeighbors(nid)
        self.pipe_neighbors = [None,None,None,None]
        self.closed_pipes = [False,False,False,False]
    
    def adjust_neighbors(self,nid2valve):
        self.valve_neighbors.adjust_neighbors(nid2valve,self.node_type)
        self.node_neighbors.adjust_neighbors(self.valve_neighbors)
        self.pipe_neighbors = self.create_pipe_neighbors()
    
    def create_pipe_neighbors(self):
        pipe_neighbors = [None,None,None,None]
        v_neighbors = self.get_valve_neighbors()
        for i,v in enumerate(v_neighbors):
            if v:
                pipe_neighbors[i] = v.pid
            else:
                self.closed_pipes[i] = True
        return pipe_neighbors
    
    def get_node_neighbor_ids(self):
        return self.node_neighbors.get_neighbors()
    
    def get_valve_neighbors(self):
        return self.valve_neighbors.get_neighbors()
    
    def get_pipe_neighbor_ids(self):
        return self.pipe_neighbors
    
    def degree(self):
        d = 4-sum(self.closed_pipes)
        return d
        
    def close_pipe(self,pid2close):
        op = None
        for i, pid in enumerate(self.pipe_neighbors):
            if pid ==  pid2close:
                self.closed_pipes[i] = True
                op = i
                break
        if op == None:
            raise(f"can not close pipe {pid2close}, not linked to this node")
            
        self._close_pipe_on_other_end()
            
    def _close_pipe_on_other_end(self,op):
        other_end_node = self.node_neighbors[op]
        if op == 0:
            other_end_node.closed_pipes[1] = True
        elif op == 1:
            other_end_node.closed_pipes[0] = True
        elif op == 2:
            other_end_node.closed_pipes[3] = True
        elif op == 3:
            other_end_node.closed_pipes[2] = True
        
            
        
        
    def __str__(self):  
        return f"nid: {self.nid}, ntype: {self.node_type},\
        nneighbors: {self.node_neighbors},\
        pneighbors: {self.pipe_neighbors},\
        vneighbors: {self.valve_neighbors},\
        degree: {self.degree()}" 

    
def get_nid2nodes(grid_size,nid2v_dict):
    nid2nodes = {}
    type_checker = NodeTypeChecker(grid_size)
    N = grid_size*grid_size
    for nid in range(N):
        node = Node(nid,type_checker,grid_size)
        node.adjust_neighbors(nid2v_dict)
        nid2nodes[nid] = node     
    return nid2nodes 

In [99]:
def choose_pid2close(node,pid_blacklist):
    valid_pids = []
    pids = [pid for pid in node.pipe_neighbors if (pid != None and pid not in pid_blacklist) ]
    print (node.pipe_neighbors,pids)
    if len(pids):
        return np.random.choice(pids,1)
    else:
        return None
    

In [117]:
grid_size = 3
contraction_ratio = 0.2
mesh = Mesh(grid_size)

nids2identify = get_nids2identify(grid_size,contraction_ratio)
identify_pairs_op =  get_identify_pairs_op(nids2identify,mesh.nid2nodes)
mesh.perform_contractions(identify_pairs_op)
mesh.update_degree_distribution()
print(mesh.contraction_reg)

npairs: [(<__main__.Node object at 0x7fc125d50160>, <__main__.Node object at 0x7fc125d50ac8>)], vpairs: [(<scripts.valves.Valve object at 0x7fc125b60dd8>, <scripts.valves.Valve object at 0x7fc125b60eb8>)],        trivial_pids: [1],contracted_nids: {1, 2}


In [119]:
choose_pid2close(mesh.nid2nodes[0],mesh.bfs_edges)

[None, 0, None, 6] []


In [120]:
len(mesh.bfs_edges)

12